In [143]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
import operator
import math 
import re
import numpy as np 

stop_words = set(stopwords.words('spanish') + list(punctuation))
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def getFile(fname):
    try:
        with open(fname, 'r', encoding='utf-8') as f:
            text = f.read()
        return text
    except Exception as e:
        print(f"Error reading the file: {e}")
        return None

        
def clean2(textoTokenizado):
    newTokens = []
    for token in textoTokenizado:
        pattern = r"[^a-zA-Z0-9 \n.,()áéóíúÁÉÍÓÚ]"

        stripped = re.sub(pattern, "", token)

        # Change any whitespace to one space
        stripped = ' '.join(stripped.split())

        # Remove start and end white spaces
        stripped = stripped.strip()
        
        newTokens.append(stripped)
    return newTokens

def clean(textoTokenizado):
    newTokens = []
    for token in textoTokenizado:
        pattern = r"[^a-zA-Z0-9 \n]"

        stripped = re.sub(pattern, "", token)

        # Change any whitespace to one space
        stripped = ' '.join(stripped.split())

        # Remove start and end white spaces
        stripped = stripped.strip()
        
        newTokens.append(stripped)
    return newTokens

def global_frequency(txtClean):

    freq_table = {}
    text = ' '.join(txtClean) #join the cleaned sentences to get the text 
    words = word_tokenize(text)
    for word in words:
        word = word.lower()
        word = ps.stem(word)
        if word not in stop_words:
            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1
                
    return freq_table

def get_keywords(txtClean, n):

    freq_table = global_frequency(txtClean)
    #sort in descending order
    freq_table_sorted = sorted(freq_table.items(), key = operator.itemgetter(1), reverse = False) 
    #print(freq_table_sorted)
    keywords = []
    for i in range(0, n):  #taking first 5 most frequent words
        keywords.append(freq_table_sorted[i][0])
    return keywords

def contar_diccionarios_con_palabra(matriz, palabra):
    contador = 0
    for i in range(1, matriz.shape[0]):
        temp = matriz[i, 2]
        if palabra in temp:
            contador += 1
    return contador


def frequency(sentences):
    A = np.empty((len(sentences) + 1, 6), dtype=object)
    for i, sent in enumerate(sentences, start=1):
        words = sent.split()
        count = len(words)
        # numero de palabras por oracion
        A[i, 0] = count
        #frecuencia de palabra en oracion
        A[i, 1] = {}
        #TF de palabra en oracion
        A[i, 2] = {}
        
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word not in stop_words:
                if word in  A[i, 1]:
                     A[i, 1][word] += 1
                else:
                     A[i, 1][word] = 1
    
        for word, freq in  A[i, 1].items():
             A[i, 2][word] = freq / count
    
    #se calcula el idf
    for i, sent in enumerate(sentences, start=1):
        A[i, 3] = {}
        for word in A[i, 2]:
            count = contar_diccionarios_con_palabra(A, word)
            #print(word, len(sentences)+1, '/', count, ' = ',len(sentences)+1 / count)
            A[i, 3][word] = math.log(len(sentences)+1 / count)

    # se calcula la TF-IDF
    for i, sent in enumerate(sentences, start=1):
        A[i, 4] = {}
        for word in A[i, 2]:
            A[i, 4][word] = A[i, 2][word] * A[i, 3][word]
            
    return A

def weigh_keywords2(txtClean, A, n):
    keywords = get_keywords(txtClean, n)
    for i in range(1, A.shape[0]):
        for word in A[i, 4]:
            if word in keywords:
                A[i, 4][word] =  A[i, 4][word] * 2
    return A

def get_sent_score2(A):
    for i in range(1, A.shape[0]):
        A[i, 5] = 0
        for word in A[i, 4]:
            A[i, 5] =  A[i, 5] + A[i, 4][word]
    return A

def get_summary2(oraciones, A):
    summary = []
    prom = np.mean(A[1:, 5])
    for i in range(1, A.shape[0]):
        if A[i, 5] > prom:
            summary.append(oraciones[i-1])
    summary = ' '.join(summary)
    return summary


In [146]:
def resumir(file, n):
    texto = getFile(file)
    print(len(texto))
    for i in range(0, n):
        textoTokenizado = sent_tokenize(texto)
        clean1 = clean2(textoTokenizado)
        txtClean = clean(clean1)
        datos = frequency(txtClean)
        datos_modificados = weigh_keywords2(txtClean, datos, 7)
        sentence_info = get_sent_score2(datos_modificados)
        summary = get_summary2(textoTokenizado, sentence_info)
        print(len(summary))
        texto = summary
    return summary
resumen = resumir('texto.txt', 1)

print(resumen)


9895
5121
El futuro de la IA: hacia inteligencias artificiales realmente inteligentes
Ramón López de Mántaras
Instituto de Investigación en Inteligencia Artificial (IIIA), Bellaterra, España
Este capítulo contiene algunas reflexiones sobre inteligencia artificial (IA). En primer lugar, se explica la distinción entre la IA fuerte y la débil, así como los conceptos relacionados de IA general y específica, dejando claro que todas las manifestaciones existentes de IA son débiles y específicas. A continuación se aborda la necesidad de proporcionar a las máquinas conocimientos de sentido común que hagan posible avanzar hacia el ambicioso objetivo de construir IA de tipo general. A lo largo de los últimos siglos, este afán por construir máquinas inteligentes nos ha conducido a inventar modelos o metáforas del cerebro humano. Por ejemplo, en el siglo XVII, Descartes se preguntó si un complejo sistema mecánico compuesto de engranajes, poleas y tubos podría, en principio, emular el pensamiento. 

In [ ]:
#ps = PorterStemmer()


def resumir(file, n):
    texto = getFile(file)
    for i in range(0, n):
        textoTokenizado = sent_tokenize(texto)
        clean1 = clean2(textoTokenizado)
        txtClean = clean(clean1)
        TFIDF_scores,txtDic = frequency(txtClean)
        TFIDF_scores = weigh_keywords(txtClean, TFIDF_scores, 7)
        sentence_info = get_sent_score(TFIDF_scores, clean1, txtDic)
        #print(sentence_info)
        summary = get_summary(sentence_info)
        print(summary)
        #print(len(texto))
        #print(len(summary))
        texto = summary
resumir('texto.txt', 1)

[[None None None None None None]
 [32
  {'futuro': 1, 'ia': 2, 'hacia': 1, 'inteligencia': 3, 'artificial': 1, 'realment': 1, 'inteligent': 1, 'ramn': 1, 'lpez': 1, 'mntara': 1, 'instituto': 1, 'investigacin': 1, 'artifici': 2, 'iiia': 1, 'bellaterra': 1, 'espaa': 1, 'est': 1, 'captulo': 1, 'contien': 1, 'alguna': 1, 'reflexion': 1, 'sobr': 1}
  {'futuro': 0.03125, 'ia': 0.0625, 'hacia': 0.03125, 'inteligencia': 0.09375, 'artificial': 0.03125, 'realment': 0.03125, 'inteligent': 0.03125, 'ramn': 0.03125, 'lpez': 0.03125, 'mntara': 0.03125, 'instituto': 0.03125, 'investigacin': 0.03125, 'artifici': 0.0625, 'iiia': 0.03125, 'bellaterra': 0.03125, 'espaa': 0.03125, 'est': 0.03125, 'captulo': 0.03125, 'contien': 0.03125, 'alguna': 0.03125, 'reflexion': 0.03125, 'sobr': 0.03125}
  {'futuro': 3.912023005428146, 'ia': 3.8926049195710446, 'hacia': 3.901972669574645, 'inteligencia': 3.8940853047414787, 'artificial': 3.901972669574645, 'realment': 3.912023005428146, 'inteligent': 3.89385903480047

ZeroDivisionError: division by zero

In [ ]:
import re
text = "This is a string with special characters: !@#$%^&*()_+`-=[]{};:'\\|,.<>/?"
# Define a regular expression pattern that matches special characters
pattern = r"[^a-zA-Z0-9 ]"
result = re.sub(pattern, "", text)
print(result)


This is a string with special characters 
